Bring Long term persistence memory to llm using redis

In [3]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory

In [4]:
llm = OllamaLLM(model="llama3.2")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant who's good at {ability}. Respond in 20 words or fewer."
         ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
base_chain = prompt | llm

In [5]:
REDIS_URL = "redis://localhost:6379/0"
store = {}

In [6]:
def get_message_history(chat_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(
        url=REDIS_URL,
        session_id=chat_id,)
redis_chain = RunnableWithMessageHistory(
    base_chain,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [10]:

redis_chain.invoke({"ability": "math", "input": "What does cosine mean?"},
                   config={"configurable": {"session_id": "user_123"}})

'Cosine is a trigonometric ratio that relates the length of sides in a right-angled triangle to angles.'

In [7]:
redis_chain.invoke({"ability": "math", "input": "Tell me more!"},
                   config={"configurable": {"session_id": "user_123"}})

'Cosine has a wide range of applications, including trigonometry, physics (e.g., waves, vectors), and signal processing.'

In [13]:

redis_chain.invoke({"ability": "physic", "input": "What is theory of relativity?"},
                   config={"configurable": {"session_id": "user_456"}})

"Albert Einstein's groundbreaking theory, describing time dilation & space contraction, revolutionizing our understanding of space, time, and gravity."

In [8]:
redis_chain.invoke({"ability": "math", "input": "Tell me more!"},
                   config={"configurable": {"session_id": "user_456"}})

'Would you like to know about time dilation (time slowing down near massive objects) and gravitational redshift effects?'